## （Optional）Create different policies for transfer system.

As much as you can to use the already implemented search agent. You just need to define the **is_goal()**, **get_successor()**, **strategy()** three functions. 

> a.	Define different policies for transfer system. 

> b.	Such as Shortest Path Priority（路程最短优先）, Minimum Transfer Priority(最少换乘优先), Comprehensive Priority(综合优先)

> c.	Implement Continuous transfer. Based on the Agent you implemented, please add this feature: Besides the @param start and @param destination two stations, add some more stations, we called @param by_way, it means, our path should from the start and end, but also include the  @param by_way stations. 

e.g 
```
1. Input:  start=A,  destination=B, by_way=[C] 
    Output: [A, … .., C, …. B]
2. Input: start=A, destination=B, by_way=[C, D, E]
    Output: [A … C … E … D … B]  
    # based on your policy, the E station could be reached firstly. 
```

In [ ]:
import requests
import re
import numpy as np
r = requests.get('http://map.amap.com/service/subway?_1469083453978&srhdata=1100_drw_beijing.json')
r.text

In [2]:
def get_lines_stations_info(text):
    '''
    根据爬取文本数据
    获取所有线路信息和所有站点信息
    
    Param
    ----------
    text : 文本信息
    
    Return
    ----------
    lines_info ： 所有线路信息的dict：key：线路名称；value：站点名称list
    stations_info ： 所有站点信息的dict：key：站点名称；value：站点坐标(x,y)
    
    '''
    lines_list = text.split("st")[1:]
    
    # 遍历text格式数据，组成地点数据结构
    # 所有线路信息的dict：key：线路名称；value：站点名称list
    lines_info = {}
    
    # 所有站点信息的dict：key：站点名称；value：站点坐标(x,y)
    stations_info = {}
        
    for i in range(len(lines_list)):
        
        # 站点名称 list
        stations = re.findall('\"n\":\"(\w+)\"', lines_list[i])
        # 站点坐标 list
        x_y = re.findall('\"sl\":\"(\d+.\d+),(\d+.\d+)\"', lines_list[i])
        x_y = [tuple(map(float, i)) for i in x_y]
        # 线路名称 
        sub_name = re.findall('\"ln\":\"(\w+)\"', lines_list[i])[0]
        
        lines_info[sub_name] = stations
        
        for i, k in dict(zip(stations, x_y)).items():
            stations_info[i] = k
    
    return lines_info, stations_info

lines_info, stations_info = get_lines_stations_info(r.text)

In [3]:
from collections import defaultdict

def get_neighbor_info(lines_info):
    '''
    根据线路信息，建立站点邻接表dict
    
    Param
    ----------
    lines_info ： 所有线路信息的dict：key：线路名称；value：站点名称list
    
    Return
    ----------
    neighbor_info ： 所有站点的邻接表
    
    '''
    neighbor_info = defaultdict(list)

    # 把str2加入str1站点的邻接表中
    def add_neighbor_dict(info, str1, str2):
        
        info[str1].append(str2)

    for sub_name, stations in lines_info.items():
        
        sta_len = len(stations)
        for i in range(sta_len -1):
            add_neighbor_dict(neighbor_info, stations[i], stations[i + 1])
            add_neighbor_dict(neighbor_info, stations[sta_len-1-i], stations[sta_len-2-i])

    return neighbor_info

neighbor_info = get_neighbor_info(lines_info)         

In [4]:
def station2sub(station):
    '''
    根据站点名称获取其所属的线路名称

    Param
    ----------
    station ： 站点名称
    
    Return
    ----------
    sub_list ： 所属线路列表（换乘站点或单站点）
    
    '''
    sub_list = []
    for sub_name, stations in lines_info.items():
        if station in stations:
            sub_list.append(sub_name)
    return sub_list


def result_text(path):
    '''
    根据搜索路径格式化输出出行路线提示信息
    
    Param
    ----------
    path ： 搜索路径
    
    Return
    ----------
    text ： 地铁出行路线提示信息
    
    '''
    
    # path中各个站点所属线路的list
    names_list = []
    
    # 遍历路径上的站点
    for index, station in enumerate(path[:-1]):
        
        name_list = station2sub(station)
        for name in name_list:
            if name in station2sub(path[index+1]):
                names_list.append(name)
           
        
    text = ""
    for index, name in enumerate(names_list):
        if index==0:
            text += f"{name}{path[0]}上车，"
        else:
            if name == names_list[index-1]: continue
            text += f"{path[index]}下车，换乘{name},"
            
    
    return text + path[-1] + "下车。" 

In [5]:
import math
def get_distance_of_path(path):
    '''
    统计完成path需要的距离
    
    '''
    distance = 0

    for i, station in enumerate(path[:-1]):
        x_y = stations_info[station]
        x_y_next = stations_info[path[i+1]]
        distance += math.hypot(x_y[0] - x_y_next[0], x_y[1] - x_y_next[1])

    return distance 

def get_change_nums_of_path(path):
    '''
    统计完成path需要的换乘的次数
    
    '''
    count = 0
    
    # path中各个站点所属线路的list
    names_list = []
    
    # 遍历路径上的站点
    for index, station in enumerate(path[:-1]):
        
        name_list = station2sub(station)
        for name in name_list:
            if name in station2sub(path[index+1]):
                names_list.append(name)
                
    for index, sub_name in enumerate(names_list):
        if index == 0: continue
        if sub_name != names_list[index-1]:
            count +=1
            
    return count

In [6]:
def get_path(from_station, to_station, neighbor_info, search_strategy=None):
    '''
    根据不同搜索策略获取不同的最终路径
    
    Param
    ----------
    from_station ： 起始站点
    to_station ： 到达站点
    neighbor_info ：地铁线路各站点邻接图
    search_strategy ：搜索策略，默认无
    
    Return
    ----------
    new_path ： 所有路径
    
    ''' 

    pathes = [[from_station]]
    visited = set()
    
    while pathes:

        path = pathes.pop(0)
        node = path[-1]
        
        if node in visited: continue                    # 若已遍历，跳过 （图中是否为环）
        
        next_stas = neighbor_info[node]
        
        for next_sta in next_stas:
            if next_sta == path[-1]: continue           # 是否为前一个站点 
            
            new_path = path + [next_sta]
            if next_sta == to_station:
                return new_path
            
            pathes.append(new_path)
        '''
        search_strategy
        搜索策略默认   
        
        A : 路程最短优先
        B : 最少换乘优先
        C : 综合优先
        
        '''
        
        if search_strategy == 'A':
            
            pathes = sorted(pathes, key=get_distance_of_path)
            
        elif search_strategy == 'B':
            
            pathes = sorted(pathes, key=get_change_nums_of_path)
            
        else:
            
            pathes = sorted(pathes, key=get_change_nums_of_path)
            pathes = sorted(pathes, key=get_distance_of_path)   
            
        visited.add(node)
        


##### 5.	Test your result with commercial applications. 

将你的结果和高德地图或者百度地图进行比较，如果有不同，请分析原因

In [19]:
path = get_path("金安桥", "建国门", neighbor_info, "A")
result_text(path)

'6号线金安桥上车，慈寿寺下车，换乘10号线,公主坟下车，换乘1号线,建国门下车。'

In [20]:
path = get_path("金安桥", "建国门", neighbor_info, "B")
result_text(path)

'6号线金安桥上车，朝阳门下车，换乘2号线,建国门下车。'

In [18]:
path = get_path("金安桥", "建国门", neighbor_info, "C")
result_text(path)

'6号线金安桥上车，慈寿寺下车，换乘10号线,公主坟下车，换乘1号线,建国门下车。'